Part 1

In [280]:
import pandas as pd
import re
import numpy as np
import copy

In [252]:
def str_int(s):
    return list(map(int, s.strip().split(' ')))

In [253]:
# file = "test.dat"
file = "day.dat"
lines = open(file,'r').read().split('\n')
seeds = pd.DataFrame({'seed':str_int(lines[0].split(':')[1])})
map_types = ['seed-to-soil map:','soil-to-fertilizer map:',
             'fertilizer-to-water map:','water-to-light map:',
             'light-to-temperature map:','temperature-to-humidity map:',
             'humidity-to-location map:']

df = pd.DataFrame(columns=['map_from','map_to','destination','source','length'])
for l in lines[1:]:
    if l in map_types:
        match = re.match(r'^(\w+)\-to\-(\w+)', l)
        map_from, map_to = match.group(1), match.group(2)
    elif l != '':
        dest,src,length = str_int(l)
        df = pd.concat([df,pd.DataFrame({'map_from':map_from, 'map_to':map_to,
                                         'destination':dest, 'source':src,
                                         'length':length}, index=[0])])
df = df.reset_index(drop=True)   
df.head()

,map_from,map_to,destination,source,length
0,seed,soil,2211745924,1281207339,39747980
1,seed,soil,3648083739,2564129012,145170114
2,seed,soil,4171944574,2333022880,44675857
3,seed,soil,540694760,848661020,78793182
4,seed,soil,256996824,588160543,260500477


In [254]:
seeds.head()

,seed
0,3082872446
1,316680412
2,2769223903
3,74043323
4,4131958457


In [255]:
from_col = 'seed'
while from_col != 'location':
    map_list = df[df['map_from'] == from_col]
    to_col = list(map_list['map_to'])[0]
    seeds[to_col] = seeds[from_col]
    for i, s in enumerate(seeds[from_col]):
        for _, m in map_list.iterrows():
            if m['source'] <= s < m['source'] + m['length']:
                seeds.at[i, to_col] = m['destination'] + s - m['source']
    from_col = to_col

seeds.head()

,seed,soil,fertilizer,water,light,temperature,humidity,location
0,3082872446,3146600596,3070093545,1126696133,2291994928,1972007158,1311845980,1085436433
1,316680412,529355900,2791332414,847935002,1268090341,64848467,84470622,84470622
2,2769223903,4276545208,4187659549,2037385377,2943142680,1678956429,1018795251,797022648
3,74043323,2119458269,1459707245,4064254365,3341541755,2961582857,4109588370,3040934752
4,4131958457,2994950048,2994950048,1051552636,2216851431,2473940599,2473940599,2473940599


In [256]:
#answer is 84470622
seeds['location'].min()

84470622

Part 2

In [343]:
file = "test.dat"
# file = "day.dat"
lines = open(file,'r').read().split('\n')
raw_seeds = str_int(lines[0].split(':')[1])
seeds = pd.DataFrame({'source':raw_seeds[::2], 'length':raw_seeds[1::2]})
map_types = ['seed-to-soil map:','soil-to-fertilizer map:',
             'fertilizer-to-water map:','water-to-light map:',
             'light-to-temperature map:','temperature-to-humidity map:',
             'humidity-to-location map:']
df = pd.DataFrame(columns=['map_from','map_to','destination','source','length'])
for l in lines[1:]:
    if l in map_types:
        match = re.match(r'^(\w+)\-to\-(\w+)', l)
        map_from, map_to = match.group(1), match.group(2)
    elif l != '':
        dest,src,length = str_int(l)
        df = pd.concat([df,pd.DataFrame({'map_from':map_from, 'map_to':map_to,
                                         'destination':dest, 'source':src,
                                         'length':length}, index=[0])])
df = df.reset_index(drop=True)   
df

,map_from,map_to,destination,source,length
0,seed,soil,50,98,2
1,seed,soil,52,50,48
2,soil,fertilizer,0,15,37
3,soil,fertilizer,37,52,2
4,soil,fertilizer,39,0,15
5,fertilizer,water,49,53,8
6,fertilizer,water,0,11,42
7,fertilizer,water,42,0,7
8,fertilizer,water,57,7,4
9,water,light,88,18,7


In [344]:
seeds

,source,length
0,79,14
1,55,13


In [306]:
# map_names = pd.concat([df['map_from'],df['map_to']]).unique()
# best = {key: np.inf for key in map_names}
# current = {key: np.inf for key in map_names}

# i=0
# for s_start, s_end in zip(seeds['seed_start'], seeds['seed_range']):
#     i+=1
#     print(i)
#     seed_range = range(s_start, s_start+s_end)
#     for s in seed_range:
#         current['seed'] = s
#         from_col = 'seed'
#         while from_col != 'location':
#             map_list = df[df['map_from'] == from_col]
#             to_col = list(map_list['map_to'])[0]
#             current[to_col] = current[from_col]
#             # if s == 82:
#             #     print(from_col, to_col)
#             #     print(map_list)
#             for _, m in map_list.iterrows():
#                 if m['source'] <= s < m['source'] + m['length']:
#                     current[to_col] = m['destination'] + s - m['source']
#                     # if s == 82:
#                     #     print("hit", from_col, to_col, m['destination'], s, m['source'], m['length'])
#             from_col = to_col
#             if current['location'] < best['location']: 
#                 best = copy.deepcopy(current)
#         # if s == 82:
#         #     print(current)
# best

seed number 82  
soil 84  
fertilizer 84   
water 84   
light 77  
temperature 45   
humidity 46  
location 46  

In [360]:
# let's get the endpoints
map_names = list(pd.concat([df['map_from'],df['map_to']]).unique())[::-1]
range_list = [0,np.inf]
for m in map_names:
    range_vals = df[df['map_from'] == m][['source','length']].sort_values(by='source')
    for _, row in range_vals.iterrows():
        a, b = row['source'], row['source']+row['length']
        range_list.extend([a,b,a-1,b-1])
    range_list = sorted(list(set(range_list)))
    print(m, range_list)
range_vals = seeds[['source','length']].sort_values(by='source')
for index, row in range_vals.iterrows():
    a = row['source']
    b = row['source']+row['length']
    range_list.extend([a,b,a-1,b-1,a+1,b+1])
range_list = sorted(list(set(range_list)))
print(range_list)


location [0, inf]
humidity [0, 55, 56, 92, 93, 96, 97, inf]
temperature [-1, 0, 55, 56, 68, 69, 70, 92, 93, 96, 97, inf]
light [-1, 0, 44, 45, 55, 56, 63, 64, 68, 69, 70, 76, 77, 92, 93, 96, 97, 99, 100, inf]
water [-1, 0, 17, 18, 24, 25, 44, 45, 55, 56, 63, 64, 68, 69, 70, 76, 77, 92, 93, 94, 95, 96, 97, 99, 100, inf]
fertilizer [-1, 0, 6, 7, 10, 11, 17, 18, 24, 25, 44, 45, 52, 53, 55, 56, 60, 61, 63, 64, 68, 69, 70, 76, 77, 92, 93, 94, 95, 96, 97, 99, 100, inf]
soil [-1, 0, 6, 7, 10, 11, 14, 15, 17, 18, 24, 25, 44, 45, 51, 52, 53, 54, 55, 56, 60, 61, 63, 64, 68, 69, 70, 76, 77, 92, 93, 94, 95, 96, 97, 99, 100, inf]
seed [-1, 0, 6, 7, 10, 11, 14, 15, 17, 18, 24, 25, 44, 45, 49, 50, 51, 52, 53, 54, 55, 56, 60, 61, 63, 64, 68, 69, 70, 76, 77, 92, 93, 94, 95, 96, 97, 98, 99, 100, inf]
[-1, 0, 6, 7, 10, 11, 14, 15, 17, 18, 24, 25, 44, 45, 49, 50, 51, 52, 53, 54, 55, 56, 60, 61, 63, 64, 67, 68, 69, 70, 76, 77, 78, 79, 80, 92, 93, 94, 95, 96, 97, 98, 99, 100, inf]
